In [1]:
from flask import Flask, jsonify, request
import pickle
import nltk
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import ast
import numpy as np
import json

In [2]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas numpy emoji

Note: you may need to restart the kernel to use updated packages.


In [4]:
emoji_dict = {
    '😀': 'feliz ',
    '😂': 'risos ',
    '😔': 'triste ',
    '👏': 'palmas ',
    '🥰': 'amável ',
    '💙': 'coração azul ',
    '🙏🏼': 'orando ',
    '✨': 'brilhando ',
    '🤮': 'nojo ',
    '🚀': 'foguete ',
    '👿': 'diabo ',
    '🤢': 'nojo ',
    '🔥': 'fogo ',
    '😡': 'fúria ',
    '😠': 'raiva ',
    '🤣': 'rindo ',
    '😃': 'feliz ',
    '😎': 'curtindo ',
    '😊': 'feliz ',
    '🤩': 'maravilhado ',
    '😋': 'delicioso ',
    '😆': 'risada ',
    '😌': 'calmo ',
    '🤔': 'pensativo ',
    '😷': 'máscara ',
    '🤣': 'muitoRiso ',
    '🥺': 'carinhoso ',
    '👍': 'positivo ',
    '🤯': 'menteExplodida ',
    '😅': 'alívio ',
    '🥰': 'carinhaComCoração ',
    '😓': 'suor ',
    '😑': 'tédio',
    '🤫': 'silêncio',
    '🤝': 'apertoDeMãos',
    '😊': 'sorriso',
    '😍': 'apaixonado',
    '😭': 'choro ',
    '🤗': 'abraço ',
    '🎉': 'festa ',
    '😎': 'descolado ',
    '😱': 'surpresa ',
    '😴': 'sono ',
    '🙌': 'celebração ',
    '🤔': 'pensativo ',
    '😘': 'beijo ',
    '🥳': 'festeiro ',
    '🙄': 'revirarOsOlhos ',
    '😌': 'alívio ',
    '🤫': 'segredo ',
    '😇': 'inocente ',
    '😂': 'muitoEngraçado ',
    '🤔': 'pensando ',
    '😴': 'sono ',
    '🤪': 'loucura ',
    '😢': 'decepcionadoAliviado ',
    '😬': 'nervoso ',
    '😌': 'alívio',
    '😔': 'triste ',
    '😞': 'desapontado ',
    '😢': 'choro ',
    '😭': 'chorando ',
    '😡': 'raiva ',
    '🤯': 'mente explodida ',
    '😳': 'surpreso ',
    '😱': 'gritando ',
    '😨': 'assustado ',
    '😴': 'sono ',
    '🥱': 'bocejando ',
    '🤢': 'enjoado ',
    '🤮': 'vomitando ',
    '🤧': 'espirro ',
    '🤒': 'doente ',
    '🤕': 'machucado ',
    '🤑': 'dinheiro ',
}

In [5]:
def emoji_to_word(textos):
    textos_processados = []
    for texto in textos:
        palavras = texto.split()
        texto_processado = []
        for palavra in palavras:
            if palavra in emoji_dict:
                texto_processado.append(emoji_dict[palavra])
            else:
                texto_processado.append(palavra)
        texto_processado = ' '.join(texto_processado)
        textos_processados.append(texto_processado)
    
    return textos_processados


In [6]:
def processarTexto(textos):
    textos_processados = []
    for texto in textos:
        texto = texto.lower()
        tokens = word_tokenize(texto)
        stop_words = [
            '@', 'banco', 'btg', 'brg', 'pactual', 'btgpactual', 'pq', 'q', 'pra', 'vcs', 'vc', 'i', 'p', 'kkk', 'y', 'of',
            'n', 'a', 'à', 'as', 'o', 'os', 'e', 'aos', 'do', 'das', 'dos', 'das', 'de', 'deles', 'dela', 'deles', 'delas',
            'para', 'que', 'em', 'algo', 'algum', 'alguma', 'alguns', 'algumas', 'aqui', 'aquele', 'aquela', 'aqueles',
            'aquelas', 'aqui', 'aquilo', 'cá', 'com', 'como', 'cada', 'coisa', 'daquele', 'daquela', 'daquilo', 'daqueles',
            'daquelas', 'desse', 'deste', 'dessa', 'desses', 'destes', 'destas', 'ele', 'eles', 'ela', 'elas', 'eu', 'nos',
            'nós', 'vocês', 'voces', 'enquanto', 'era', 'está', 'estamos', 'estão', 'estar', 'estará', 'estive', 'estivemos',
            'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem',
            'estivéssemos', 'estiveste', 'estivestes', 'estou', 'fará', 'farta', 'farto', 'fez', 'fim', 'foi', 'fomos',
            'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui',
            'fôssemos', 'há', 'houve', 'hoje', 'isso', 'isto', 'já', 'lá', 'lhe', 'lhes', 'lo', 'logo', 'mas', 'me', 'mesma',
            'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'minha', 'minhas', 'na', 'no', 'nas', 'nos', 'naquela', 'naquelas',
            'naquele', 'naqueles', 'nem', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes',
            'ninguém', 'nosso', 'nossa', 'nossos', 'nossas', 'num', 'numa', 'outra', 'outras', 'outro', 'outros', 'pela',
            'pelo', 'perante', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'própria', 'próprio',
            'próprias', 'próprios', 'qual', 'quando', 'quanto', 'quantos', 'quantas', 'quê', 'quem', 'quer', 'quereis',
            'querem', 'queremas', 'quis', 'quisemos', 'quiser', 'quisera', 'quiseram', 'quiséramos', 'quiserem',
            'quisermos', 'quisésseis', 'quiséssemos', 'quiseste', 'quisestes', 'quiseste', 'quisestes', 'quizer',
            'quizeram', 'quizerem', 'quizermos', 'quizesse', 'quizessem', 'quizéssemos', 'são', 'se', 'seja', 'sejam',
            'sejamos', 'sem', 'sendo', 'ser', 'será', 'serão', 'será', 'seriam', 'seríamos', 'serias', 'seríeis', 'sete',
            'seu', 'seus', 'sob', 'sobre', 'sois', 'só', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'te',
            'tem', 'têm', 'temos', 'tendes', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão',
            'terá', 'teriam', 'teríamos', 'terias', 'teríeis', 'teu', 'teus', 'teve', 'tivemos', 'tiver', 'tivera',
            'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes',
            'tiveste', 'tivestes', 'um', 'uma', 'umas', 'uns'
        ]
        tokens = [
            token for token in tokens if token not in stop_words and not token.startswith('@') and token.isalpha()
        ]
        textos_processados.append(tokens)
    return textos_processados


In [7]:
def vetorizar_frases(frases, dictionary):
    frases = [' '.join(tokens) for tokens in frases]
    vectorizer = CountVectorizer()
    vectorizer.set_params(vocabulary=dictionary)
    frases_vetorizadas = vectorizer.fit_transform(frases)
    return frases_vetorizadas

In [8]:
with open('modelo_naive_bayes.pkl', 'rb') as file:
    modelo_carregado = pickle.load(file)

In [9]:
# Carregar o dicionário usando pickle
with open('dictionary.pkl', 'rb') as file:
    dictionary2 = pickle.load(file)

In [10]:
app = Flask(__name__)

In [11]:
import numpy as np

@app.route('/classificar', methods=['POST'])
def classificar():
    dados = request.json
    # Aplique a função emoji_to_word() aos dados do web scraping
    textos_entrada = dados["dados"]
    textos_processados = emoji_to_word(textos_entrada)
    textos_processados = processarTexto(textos_processados)
    frases_vetorizadas = vetorizar_frases(textos_processados, dictionary2)
    frases_vetorizadas = frases_vetorizadas.toarray()  # Converter para matriz densa
    predicoes = modelo_carregado.predict(frases_vetorizadas)

    # Mapear valores numéricos para palavras correspondentes
    mapeamento_classes = {0: "negativo", 1: "neutro", 2: "positivo"}
    predicoes_palavras = [mapeamento_classes[predicao] for predicao in predicoes]

    return json.dumps(predicoes_palavras)

In [12]:
from collections import Counter

@app.route('/proporcoes', methods=['POST'])
def proporcoes():
    predicoes_palavras = request.json
    predicoes_palavras = predicoes_palavras["dados"]

    # Mapear os sentimentos para os valores numéricos correspondentes
    mapeamento_sentimentos = {"negativo": 0, "neutro": 1, "positivo": 2}
    predicoes_numeros = [mapeamento_sentimentos[sentimento] for sentimento in predicoes_palavras]

    # Contar a ocorrência de cada sentimento
    proporcoes = Counter(predicoes_numeros)

    # Calcular as proporções
    total = len(predicoes_numeros)
    proporcoes = {sentimento: count/total for sentimento, count in proporcoes.items()}

    return json.dumps(proporcoes)


In [13]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

@app.route('/nuvem-palavras', methods=['POST'])
def nuvem_palavras():
    dados = request.json["dados"]
    
    # Unir todos os textos em uma única string
    texto_completo = ' '.join(dados)

    # Criar a nuvem de palavras
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_completo)

    # Plotar a nuvem de palavras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout()

    # Salvar a imagem da nuvem de palavras em um arquivo
    imagem_nuvem = 'nuvem_palavras.png'
    plt.savefig(imagem_nuvem)

    # Retornar o nome do arquivo da imagem
    return jsonify({"imagem_nuvem": imagem_nuvem})

In [15]:
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from flask import jsonify

nltk.download('punkt')
nltk.download('stopwords')

@app.route('/top-palavras', methods=['POST'])
def top_palavras():
    dados = request.json["dados"]

    # Unir todos os textos em uma única string
    texto_completo = ' '.join(dados)

    # Tokenizar o texto em palavras
    tokens = word_tokenize(texto_completo)

    # Remover stopwords das palavras tokenizadas
    stop_words = set(stopwords.words('portuguese'))
    tokens_filtrados = [token for token in tokens if token.lower() not in stop_words]

    # Contar a ocorrência das palavras
    contagem_palavras = Counter(tokens_filtrados)

    # Obter as top 10 palavras mais frequentes
    top_palavras = contagem_palavras.most_common(10)

    # Retornar as top 10 palavras em formato JSON
    return jsonify({"top_palavras": top_palavras})


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Jun/2023 17:45:14] "POST /top-palavras HTTP/1.1" 200 -
